<h1 style="color:#FF6666;text-align:center;border-bottom: 3px double #FF6666;">AI-powered Credit Card expenses categorization</h1>

Studies notebook by Lucas Antonio O. Rodrigues, phD.


In [4]:
from langchain_openai import ChatOpenAI
from langchain.llms.ollama import Ollama
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.prompts import PromptTemplate
import pandas as pd
from dotenv import main
import os
import json

main.load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

chat_model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
llm = Ollama(
    model="mistral:7b",
    verbose=False,
    temperature=0,

)

Loading bank data

In [2]:
BANK_DATA_PATH = "data/csv/October2023_3633.csv"

df = pd.read_csv(BANK_DATA_PATH)
df.fillna(0, inplace=True)

df["Posted Date"] = pd.to_datetime(df["Posted Date"])
df.head()

,Posted Date,Reference Number,Payee,Address,Amount
0,2023-10-16,28806005720001400758945,Online payment from CHK 5219,0,145.75
1,2023-10-16,24055233288400340050802,SNACK SODA VENDING CHICAGO IL,CHICAGO IL,-1.35
2,2023-10-16,24013393287001575757741,SOUTH LOOP MARKET-CERMAK CHICAGO IL,CHICAGO IL,-41.49
3,2023-10-16,24247603287500836202405,PIZANOS PIZZA AND PASTA CHICAGO IL,CHICAGO IL,-79.00
4,2023-10-14,24523923286900019573549,TOUS LES JOURS CHICAGO IL,CHICAGO IL,-27.82


Creating the Categorization parameters

In [3]:
categories = [
    "Payment",
    "Transportation",
    "Food",
    "Shopping",
    "Service",
    "Subscription",
    "Farmacy",
    "Entertainment",
    "Pets",
    "Other"
]

transactions = df["Payee"].to_json()

In [4]:
category_template = """
You will be given a list of transactions from your bank account.
Please categorize the following transactions using the rules below:
1. Use ONLY the following categories: 
{categories}
2. If you are not sure, use the categorie "Other".


---
The transactions are:
{transactions}
---
you must answer using JSON format, with the following format:

{{
    "transaction_id": "category"
}}

PLEASE ANSWER USING ONLY THE JSON FORMAT ABOVE.
"""

category_prompt_template = PromptTemplate.from_template(category_template)
prompt = category_prompt_template.format(transactions=transactions, categories=categories)


In [5]:
response = chat_model.invoke(prompt)
data_categories = json.loads(response.content)
# response = llm(prompt)
# data_categories = json.loads(response)
df_categorized = df.join(pd.Series(data_categories,name="category").reset_index(drop=True))
df_categorized.head()

,Posted Date,Reference Number,Payee,Address,Amount,category
0,2023-10-16,28806005720001400758945,Online payment from CHK 5219,0,145.75,Payment
1,2023-10-16,24055233288400340050802,SNACK SODA VENDING CHICAGO IL,CHICAGO IL,-1.35,Food
2,2023-10-16,24013393287001575757741,SOUTH LOOP MARKET-CERMAK CHICAGO IL,CHICAGO IL,-41.49,Food
3,2023-10-16,24247603287500836202405,PIZANOS PIZZA AND PASTA CHICAGO IL,CHICAGO IL,-79.00,Food
4,2023-10-14,24523923286900019573549,TOUS LES JOURS CHICAGO IL,CHICAGO IL,-27.82,Food


In [6]:
df_categorized.drop(df_categorized.query("Amount > 0").index, inplace=True)
df_categorized['Amount'] = df_categorized['Amount'].abs()
df_categorized.head()

,Posted Date,Reference Number,Payee,Address,Amount,category
1,2023-10-16,24055233288400340050802,SNACK SODA VENDING CHICAGO IL,CHICAGO IL,1.35,Food
2,2023-10-16,24013393287001575757741,SOUTH LOOP MARKET-CERMAK CHICAGO IL,CHICAGO IL,41.49,Food
3,2023-10-16,24247603287500836202405,PIZANOS PIZZA AND PASTA CHICAGO IL,CHICAGO IL,79.00,Food
4,2023-10-14,24523923286900019573549,TOUS LES JOURS CHICAGO IL,CHICAGO IL,27.82,Food
5,2023-10-10,24055233283400345058524,SNACK SODA VENDING CHICAGO IL,CHICAGO IL,2.10,Food


In [7]:
agent = create_pandas_dataframe_agent(
#     llm=Ollama(
#     model="llama2",
#     verbose=False,
#     temperature=0,

# ),
    llm=chat_model,
    df=df_categorized,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_executor_kwargs={"handle_parsing_errors":True}
)

# Generating meaningful insights

Creating Pydantic data models to handle model output format

In [8]:
from typing import List,Union
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain.pydantic_v1 import BaseModel, Field, validator

In [9]:
CHART_TYPES = ['bar', 'line', 'pie', 'scatter', 'area', 'doughnut','table']

class Chart(BaseModel):
    title: str = Field(description="The title of the chart")
    type: str = Field(description="The type of the chart. should be one of the 'CHART_TYPES'")
    labels: List[str] = Field(description="The labels of the chart")
    datasets: List[List[float]] = Field(description="The datasets of the chart. Should be one list of floats for each label in the labels list")

    @validator('type')
    def type_must_be_valid(cls, v):
        if v is None:
            return v
        if v not in CHART_TYPES:
            raise ValueError(f"type must be one of the following: {CHART_TYPES}")
        return v
    

class InsightResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    chart: Union[Chart,None] = Field(description="The chart of the answer")

class SqlChart(BaseModel):
    type: str = Field(description="The type of the chart. should be one of the 'CHART_TYPES'")
    title: str = Field(description="The title of the chart")
    labels: List[str] = Field(description="The labels of the chart")

    @validator('type')
    def type_must_be_valid(cls, v):
        if v is None:
            return v
        if v not in CHART_TYPES:
            raise ValueError(f"type must be one of the following: {CHART_TYPES}")
        return v

In [10]:
pydantic_output_parser = PydanticOutputParser(pydantic_object=InsightResponse)

format_instructions = pydantic_output_parser.get_format_instructions()

new_parser = OutputFixingParser.from_llm(parser=pydantic_output_parser,llm=chat_model)

In [11]:
insight_template = """
Answer the following query:
{query}
Your answer should follow the following rules:
1. the answer to the query should be a text and a chart or a table.
2. You should decide if the answer should be a chart or a table.
3. The chart can be one of the following types: {chart_types}.
4. If you can't build a chart of the types of item 3, leave chart as "None".
5. Use ONLY the format instructions below to format your answer.

{format_instructions}
"""

query = "What category did I spend the most money on?"

insight_prompt_template = PromptTemplate.from_template(insight_template)
prompt = insight_prompt_template.format(query=query, chart_types=CHART_TYPES, format_instructions=format_instructions)

insight_response = agent.run(prompt)
try:
    insight = pydantic_output_parser.parse(insight_response)
except ValueError:
    print("Output parsing failed. Trying to fix the output...")
    insight = new_parser.parse(insight_response)
insight



> Entering new AgentExecutor chain...
Could not parse tool input: {'arguments': 'import pandas as pd\n\n# Calculate the total amount spent for each category\ncategory_spending = df.groupby(\'category\')[\'Amount\'].sum().reset_index()\n\n# Find the category with the highest spending\nmax_spending_category = category_spending.loc[category_spending[\'Amount\'].idxmax(), \'category\']\n\n# Create the answer string\nanswer = f"The category with the highest spending is {max_spending_category}."\n\n# Create the chart\nchart = {\n    "title": "Spending by Category",\n    "type": "bar",\n    "labels": category_spending[\'category\'].tolist(),\n    "datasets": [category_spending[\'Amount\'].tolist()]\n}\n\n# Format the output\noutput = {\n    "answer": answer,\n    "chart": chart\n}\n\noutput', 'name': 'python'} because the `arguments` is not valid JSON.Invalid or incomplete response
Invoking: `python_repl_ast` with `{'query': 'import pandas as pd\n\n# Calculate the total amount spent for eac

InsightResponse(answer='The category with the highest spending is Food.', chart=Chart(title='Spending by Category', type='bar', labels=['Entertainment', 'Farmacy', 'Food', 'Other', 'Payment', 'Shopping', 'Transportation'], datasets=[[94.39, 116.08, 1198.69, 1003.16, 66.71, 156.59, 86.51]]))

In [ ]:
import plotly.graph_objects as go

if insight.chart is None:
    print(insight.answer)
else:
    chart_data = insight.chart

    fig = go.Figure(data=[
        go.Bar(x=chart_data.labels, y=dataset, name=label)
        for label, dataset in zip(chart_data.labels, chart_data.datasets)
    ])

    fig.update_layout(
        title=chart_data.title,
        template="plotly_dark"
    )

    print(insight.answer)
    fig.show()


The category with the highest spending is Food.


# Using SQL Agent

In [13]:
from langchain.utilities.sql_database import SQLDatabase

DB_PATH = "data/transactions.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{DB_PATH}")

def get_schema(_):
    return db.get_table_info()

def run_query(query):
    return db.run(query.strip("`"))

In [14]:
pydantic_output_parser = PydanticOutputParser(pydantic_object=SqlChart)

format_instructions = pydantic_output_parser.get_format_instructions()

new_parser = OutputFixingParser.from_llm(parser=pydantic_output_parser,llm=llm)

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

sql_template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
sql_prompt = ChatPromptTemplate.from_template(sql_template)

natural_template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(natural_template)

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | sql_prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

sql_agent = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: run_query(x["query"]),
    )
    | prompt_response
    | llm
)

In [17]:
question = {"question": "What is the total amount of money spent on food in USD?"}

for chunk in sql_agent.stream(question):
    print(chunk, end="", flush=True)


Based on the provided table schema, question, SQL query, and SQL response, it appears that the total amount of money spent on food in USD is -3271.1999999999985. This indicates a negative balance, which may not be accurate or meaningful in this context. It's possible that there are errors or inconsistencies in the data provided.

In [ ]:
chart_template = """Based on the table schema below, question, sql query, and sql response, select a chart that would represent the 
response to the user's question, and indicate it using the format instructions below:

{format_instructions}

---
Here are the table schema, question, sql query, and sql response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}

YOUR ANSWER SHOULD INCLUDE ONLY THE JSON FORMAT INSTRUCTIONS ABOVE."""

datasets_template = """Based on the table schema, original SQL Query, and the chart json below, write a new SQL query to obtain the data for the chart:
{schema}

Original SQL Query: {query}
Chart: {chart}

your answer should be ONLY the SQL query."""

final_template = """Based on the chart json and the datasets below, write a new chart JSON:
Chart: {chart}
Datasets: {datasets}

your answer should be ONLY the chart JSON following the format instructions below:
{format_instructions}

DO NOT include any other text rather than the chart JSON."""

prompt_chart = ChatPromptTemplate.from_template(chart_template)
dataset_prompt = ChatPromptTemplate.from_template(datasets_template)
final_prompt = ChatPromptTemplate.from_template(final_template)

def get_format_instructions_for_reference_chart(_):
    pydantic_output_parser = PydanticOutputParser(pydantic_object=SqlChart)
    return pydantic_output_parser.get_format_instructions()

def get_format_instructions_for_insight_chart(_):
    pydantic_output_parser = PydanticOutputParser(pydantic_object=InsightResponse)
    return pydantic_output_parser.get_format_instructions()

chart_agent = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: run_query(x["query"]),
        format_instructions=get_format_instructions_for_reference_chart
    )
    | prompt_chart
    | llm
)

chart_response = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(chart=chart_agent)
    | RunnablePassthrough.assign(
        schema=get_schema,
        query=sql_response
    )
    | dataset_prompt
    | llm
    | StrOutputParser()
)

chart_generator = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(chart=chart_agent)
    | RunnablePassthrough.assign(new_query=chart_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        datasets=lambda x: run_query(x["new_query"].replace("sql","")),
        format_instructions=get_format_instructions_for_insight_chart
    )
    | final_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = {"question": "How much did I spent in between 1st and 30th of November per category? consider only the transactions of type 'debit'"}

agent_response = chart_generator.invoke(question)
print(agent_response)

```
{
    "type": "pie",
    "title": "Total Spent by Category",
    "labels": [
        {"category_name": "ACCOMMODATION"},
        {"category_name": "ENTERTAINMENT"},
        {"category_name": "FARMACY"},
        {"category_name": "FOOD"},
        {"category_name": "INCOME"},
        {"category_name": "OTHER"},
        {"category_name": "PETS"},
        {"category_name": "RENT"},
        {"category_name": "RESTAURANT"},
        {"category_name": "SERVICE"},
        {"category_name": "SHOPPING"},
        {"category_name": "SUBSCRIPTION"},
        {"category_name": "TRANSFER"},
        {"category_name": "TRANSPORTATION"},
        {"category_name": "UTILITIES"}
    ],
    "datasets": [
        [-791.2299999999999, "ACCOMMODATION"],
        [-6529.730000000003, "ENTERTAINMENT"],
        [-4947.27, "FARMACY"],
        [-66259.41999999982, "FOOD"],
        [-30527.130000000012, "INCOME"],
        [-78336.38999999994, "OTHER"],
        [-25232.060000000012, "PETS"],
        [-50878.54999999